In [1]:
import matplotlib
# matplotlib.use('agg')
import pandas as pd
from dipy.segment.metric import mdf
from fastdtw import dtw
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from keras.datasets import mnist
from utils.partition import *
from model.GMVAE import *
import os
from scipy.io import loadmat
os.environ["TF_CPP_MIN_LOG_LEVEL"]='2'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Using TensorFlow backend.


In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)


del_all_flags(tf.flags.FLAGS)

flags = tf.flags
PARAMETERS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')
#########################################################
## Input Parameters
#########################################################

## Dataset
flags.DEFINE_string('dataset', 'mnist', 'Specify the desired dataset (mnist)')
flags.DEFINE_integer('seed', -1, 'Random Seed')

## GPU
flags.DEFINE_integer('gpu', 1, 'Using Cuda, 1 to enable')
flags.DEFINE_integer('gpuID', 0, 'Set GPU Id to use')

## Training
flags.DEFINE_integer('batch_size', 64, 'Batch size of training data')
flags.DEFINE_integer('num_epochs', 150, 'Number of epochs in training phase')
flags.DEFINE_float('learning_rate', 1e-3, 'Learning rate for training')
flags.DEFINE_float('decay_epoch', -1,
                   'Reduces the learning rate every decay_epoch')
flags.DEFINE_float('lr_decay', 0.5, 'Learning rate decay for training')

## Architecture
flags.DEFINE_integer('num_classes', 4, 'Number of clusters')
flags.DEFINE_integer('gaussian_size', 64,
                     'Size of the gaussian learnt by the network')

## Partition parameters
flags.DEFINE_float(
    'train_proportion', 1.0,
    'Proportion of examples to consider for training only  (0.0-1.0)')
flags.DEFINE_integer('batch_size_val', 50, 'Batch size of validation data')
flags.DEFINE_integer('batch_size_test', 50, 'Batch size of test data')

## Gumbel parameters
flags.DEFINE_float(
    'temperature', 1.0,
    'Initial temperature used in gumbel-softmax (recommended 0.5-1.0)')
flags.DEFINE_integer('decay_temperature', 1,
                     'Set 1 to decay gumbel temperature at every epoch')
flags.DEFINE_integer('hard_gumbel', 0, 'Hard version of gumbel-softmax')
flags.DEFINE_float('min_temperature', 0.5,
                   'Minimum temperature of gumbel-softmax after annealing')
flags.DEFINE_float('decay_temp_rate', 0.00693,
                   'Temperature decay rate at every epoch')

## Loss function parameters
flags.DEFINE_string('loss_type', 'bce',
                    'Desired loss function to train (mse, bce)')
flags.DEFINE_float('w_gaussian', 1.0, 'Weight of Gaussian regularization')
flags.DEFINE_float('w_categorical', 1.0,
                   'Weight of Categorical regularization')
flags.DEFINE_float('w_reconstruction', 1.0, 'Weight of Reconstruction loss')

## Others
flags.DEFINE_integer('verbose', 0, "Print extra information at every epoch.")
flags.DEFINE_integer('random_search_it', 20, 'Iterations of random search')

if PARAMETERS.gpu == 1:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(PARAMETERS.gpuID)

if PARAMETERS.seed < 0:
    np.random.seed(None)
else:
    np.random.seed(PARAMETERS.seed)

#########################################################
## Read Data
#########################################################
if PARAMETERS.dataset == "mnist":
    print("Loading mnist dataset...")
    # load mnist data
    # (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train=np.load("x_scale4.npy")
    y_train=np.load("y_label4.npy")
    x_test =  np.load("test4.npy")
    y_test =  np.load("y_test4.npy")

    # x_train = x_train / 255.0
    # x_test = x_test / 255.0
else:
    raise "invalid dataset, valid datasets are: mnist"

## Set datatypes
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)

## Flatten data
x_train = flatten_array(x_train)
x_test = flatten_array(x_test)

#########################################################
## Data Partition
#########################################################
test_data, test_labels = x_test, y_test

if PARAMETERS.train_proportion == 1.0:
    train_data, train_labels, val_data, val_labels = x_train, y_train, x_test, y_test
else:
    train_data, train_labels, val_data, val_labels = partition_train_val(
        x_train, y_train, PARAMETERS.train_proportion, PARAMETERS.num_classes)

if PARAMETERS.verbose == 1:
    print("Train size: %dx%d" % (train_data.shape[0], train_data.shape[1]))
    if PARAMETERS.train_proportion < 1.0:
        print("Validation size: %dx%d" %
              (val_data.shape[0], val_data.shape[1]))
    print("Test size: %dx%d" % (test_data.shape[0], test_data.shape[1]))

#########################################################
## Train and Test Model
#########################################################
tf.reset_default_graph()
if PARAMETERS.seed > -1:
    tf.set_random_seed(PARAMETERS.seed)

## Model Initialization
gmvae = GMVAE(PARAMETERS)

## Training Phase
history_loss = gmvae.train(train_data, train_labels, val_data, val_labels)

## Testing Phase
accuracy, nmi = gmvae.test(test_data, test_labels, PARAMETERS.batch_size_test)

print("Testing phase...")
print("Accuracy: %.5lf,  NMI: %.5lf" % (accuracy, nmi))


Loading mnist dataset...


FileNotFoundError: [Errno 2] No such file or directory: 'test4.npy'

In [2]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)


del_all_flags(tf.flags.FLAGS)

flags = tf.flags
PARAMETERS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

#########################################################
## Input Parameters
#########################################################

## Dataset
flags.DEFINE_string('dataset', 'mnist', 'Specify the desired dataset (mnist)')
flags.DEFINE_integer('seed', -1, 'Random Seed')

## GPU
flags.DEFINE_integer('gpu', 1, 'Using Cuda, 1 to enable')
flags.DEFINE_integer('gpuID', 1, 'Set GPU Id to use')

## Training
flags.DEFINE_integer('batch_size', 64, 'Batch size of training data')
flags.DEFINE_integer('num_epochs', 100, 'Number of epochs in training phase')
flags.DEFINE_float('learning_rate', 1e-3, 'Learning rate for training')
flags.DEFINE_float('decay_epoch', -1,
                   'Reduces the learning rate every decay_epoch')
flags.DEFINE_float('lr_decay', 0.5, 'Learning rate decay for training')

## Architecture
flags.DEFINE_integer('num_classes', 1, 'Number of clusters')
flags.DEFINE_integer('gaussian_size', 64,
                     'Size of the gaussian learnt by the network')

## Partition parameters
flags.DEFINE_float(
    'train_proportion', 1.0,
    'Proportion of examples to consider for training only  (0.0-1.0)')
flags.DEFINE_integer('batch_size_val', 200, 'Batch size of validation data')
flags.DEFINE_integer('batch_size_test', 200, 'Batch size of test data')

## Gumbel parameters
flags.DEFINE_float(
    'temperature', 1.0,
    'Initial temperature used in gumbel-softmax (recommended 0.5-1.0)')
flags.DEFINE_integer('decay_temperature', 1,
                     'Set 1 to decay gumbel temperature at every epoch')
flags.DEFINE_integer('hard_gumbel', 0, 'Hard version of gumbel-softmax')
flags.DEFINE_float('min_temperature', 0.5,
                   'Minimum temperature of gumbel-softmax after annealing')
flags.DEFINE_float('decay_temp_rate', 0.00693,
                   'Temperature decay rate at every epoch')

## Loss function parameters
flags.DEFINE_string('loss_type', 'bce',
                    'Desired loss function to train (mse, bce)')
flags.DEFINE_float('w_gaussian', 1.0, 'Weight of Gaussian regularization')
flags.DEFINE_float('w_categorical', 1.0,
                   'Weight of Categorical regularization')
flags.DEFINE_float('w_reconstruction', 1.0, 'Weight of Reconstruction loss')

## Others
flags.DEFINE_integer('verbose', 0, "Print extra information at every epoch.")
flags.DEFINE_integer('random_search_it', 20, 'Iterations of random search')

if PARAMETERS.gpu == 1:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(PARAMETERS.gpuID)

if PARAMETERS.seed < 0:
    np.random.seed(None)
else:
    np.random.seed(PARAMETERS.seed)

#########################################################
## Read Data
#########################################################
if PARAMETERS.dataset == "mnist":
    print("Loading mnist dataset...")
    # load mnist data
    # (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # x_train=np.load("x_scale4.npy")
    # y_train=np.load("y_label4.npy")
    # x_test =  np.load("test4.npy")
    # y_test =  np.load("y_test4.npy")

    x_train=np.load("x_scale5.npy")
    y_train=np.load("y_label5.npy")
    x_test =  np.load("x_test5.npy")
    y_test =  np.load("y_test5.npy")

    # x_train = x_train / 255.0
    # x_test = x_test / 255.0
else:
    raise "invalid dataset, valid datasets are: mnist"

## Set datatypes
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)

## Flatten data
x_train = flatten_array(x_train)
x_test = flatten_array(x_test)

#########################################################
## Data Partition
#########################################################
test_data, test_labels = x_test, y_test

if PARAMETERS.train_proportion == 1.0:
    train_data, train_labels, val_data, val_labels = x_train, y_train, x_test, y_test
else:
    train_data, train_labels, val_data, val_labels = partition_train_val(
        x_train, y_train, PARAMETERS.train_proportion, PARAMETERS.num_classes)

if PARAMETERS.verbose == 1:
    print("Train size: %dx%d" % (train_data.shape[0], train_data.shape[1]))
    if PARAMETERS.train_proportion < 1.0:
        print("Validation size: %dx%d" %
              (val_data.shape[0], val_data.shape[1]))
    print("Test size: %dx%d" % (test_data.shape[0], test_data.shape[1]))

#########################################################
## Train and Test Model
#########################################################
tf.reset_default_graph()
if PARAMETERS.seed > -1:
    tf.set_random_seed(PARAMETERS.seed)

## Model Initialization
gmvae = GMVAE(PARAMETERS)

## Training Phase
history_loss = gmvae.train(train_data, train_labels, val_data, val_labels)

## Testing Phase
accuracy, nmi = gmvae.test(test_data, test_labels, PARAMETERS.batch_size_test)

print("Testing phase...")
print("Accuracy: %.5lf,  NMI: %.5lf" % (accuracy, nmi))

Loading mnist dataset...
Training phase...
(Epoch 1 / 100) Train Loss: 64.32565; Val Loss: 64.05032   Train ACC: 0.28493; Val ACC: 0.20000   Train NMI: 0.00000; Val NMI: 0.00000
(Epoch 2 / 100) Train Loss: 63.15116; Val Loss: 63.89474   Train ACC: 0.28554; Val ACC: 0.20000   Train NMI: 0.00000; Val NMI: 0.00000
(Epoch 3 / 100) Train Loss: 62.76838; Val Loss: 63.04191   Train ACC: 0.28554; Val ACC: 0.20000   Train NMI: 0.00000; Val NMI: 0.00000
(Epoch 4 / 100) Train Loss: 62.37618; Val Loss: 62.96083   Train ACC: 0.28462; Val ACC: 0.20000   Train NMI: 0.00000; Val NMI: 0.00000
(Epoch 5 / 100) Train Loss: 62.13655; Val Loss: 62.51428   Train ACC: 0.28676; Val ACC: 0.20000   Train NMI: 0.00000; Val NMI: 0.00000
(Epoch 6 / 100) Train Loss: 61.95889; Val Loss: 62.44288   Train ACC: 0.28554; Val ACC: 0.20000   Train NMI: 0.00000; Val NMI: 0.00000
(Epoch 7 / 100) Train Loss: 61.70401; Val Loss: 62.08905   Train ACC: 0.28585; Val ACC: 0.20000   Train NMI: 0.00000; Val NMI: 0.00000
(Epoch 8 / 1

In [3]:
def revScale(tra):
    lat_min= 48.90202222222222
    lat_max=49.0623
    lon_max=-123.02870277777778
    lon_min=-123.41741944444445
    lat=tra[:,0]*(lat_max-lat_min)+lat_min
    lon=tra[:,1]*(lon_max-lon_min)+lon_min
    new_tra=np.stack((lat,lon),axis=1)
    return new_tra

In [4]:
valdata=np.load("9msc.npy")
valdata = valdata.astype(np.float32)
valdata = flatten_array(valdata)

In [12]:
orst=np.load("9m353.npy")

In [13]:
# test_batch_data = test_data
# reconstructed = gmvae.reconstruct_data(test_batch_data)

In [11]:
reconstructed = gmvae.reconstruct_data(valdata)

In [22]:
ind=209

In [23]:
tra1=reconstructed[ind].reshape(50, 2)

In [24]:
new_tra1=revScale(tra1)

In [25]:
tra1_or=valdata[ind].reshape(50,2)

In [26]:
tra1_or=revScale(tra1_or)

In [27]:
pd.DataFrame(tra1_or,columns=['lat','lon']).to_csv(f"./generatetest/or_{ind}.csv")

In [28]:
pd.DataFrame(new_tra1,columns=['lat','lon']).to_csv(f"./generatetest/new_{ind}.csv")

In [13]:
mdf(tra1_or,new_tra1)

0.014685261441497273

In [45]:
wr=pd.read_csv("wr.csv",index_col=0)

In [210]:
mdf(tra1_or,new_tra1)

0.009703701828518773

In [211]:
dtw(tra1_or,new_tra1)[0]

0.4523026149061309

In [54]:
dtw(tra1_or,np.array(wr))[0]

0.46914546797847834

In [5]:
test_data.shape

(500, 100)

NameError: name 'np' is not defined

In [7]:
st=np.load("9m353.npy")

In [ ]:
valdata